## Loan Recommender

In [58]:
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import json
import ast

In [59]:
# Initialize the LLM (using Ollama in this example).
llm = Ollama(model="phi4:latest", base_url="http://127.0.0.1:11434")

In [60]:

# # Create a prompt template for extracting parameters.
# extraction_prompt = PromptTemplate(
#     input_variables=["user_input"],
#     template=(
#         "Given the following user input:\n"
#         "\"{user_input}\"\n\n"
#         "Extract the following parameters as a JSON object:\n"
#         "- deposit_amount: The amount of money deposited (if mentioned; otherwise, None).\n"
#         "- deposit_duration: The duration of the deposit in months (should be between 1 and 12; if mentioned; otherwise, None).\n"
#         "- loan_amount: The total loan amount (if mentioned; maximum allowed is 300000000; otherwise, None).\n"
#         "- credit_score: One of the following values: A, B, C, D, E, None (if mentioned; otherwise, None).\n"
#         "- number_of_installments: Allowed values are 4, 6, 9, 12, 18, 24, 36, 48 (if mentioned; otherwise, None).\n"
#         "- interest_rate: Allowed values are 0, 2, 4, 18, 23 (if mentioned; otherwise, None).\n\n"
#         "If a parameter is not mentioned in the input, set its value to None.\n\n"
#         "Example output:\n"
#         "{{\n"
#         '  "deposit_amount": 5000000,\n'
#         '  "deposit_duration": 6,\n'
#         '  "loan_amount": "None",\n'
#         '  "credit_score": "None",\n'
#         '  "number_of_installments": "None",\n'
#         '  "interest_rate": "None"\n'
#         "}}"
#     )
# )
# # Create the extraction chain.
# extraction_chain = LLMChain(llm=llm, prompt=extraction_prompt, verbose=True)

# #

In [61]:

# Create a prompt template for extracting parameters.
extraction_prompt = PromptTemplate(
    input_variables=["user_input"],
    template=(
        "Given the following user input:\n"
        "\"{user_input}\"\n\n"
        "Extract the following parameters as a JSON object:\n"
        "- deposit_amount: The amount of money deposited (if mentioned; otherwise, None).\n"
        "- deposit_duration: The duration of the deposit in months (if mentioned; otherwise, None).\n"
        "- loan_amount: The total loan amount (if mentioned; otherwise, None).\n"
        "- credit_score (if mentioned; otherwise, None).\n"
        "- number_of_installments (if mentioned; otherwise, None).\n"
        "- interest_rate (if mentioned; otherwise, None).\n\n"
        "If a parameter is not mentioned in the input, set its value to None.\n\n"
        "Example output:\n"
        "{{\n"
        '  "deposit_amount": 5000000,\n'
        '  "deposit_duration": 6,\n'
        '  "loan_amount": "None",\n'
        '  "credit_score": "None",\n'
        '  "number_of_installments": "None",\n'
        '  "interest_rate": "None"\n'
        "}}"
    )
)
# Create the extraction chain.
extraction_chain = LLMChain(llm=llm, prompt=extraction_prompt, verbose=True)

#

In [62]:

# Define the Function to Parse the LLM Output

def parse_extraction_result(result_str: str) -> dict:
    """
    Extract the dictionary from the LLM output.
    First, find the substring between the first '{' and the last '}'.
    Then try to parse that substring as JSON.
    If that fails (for example, due to single quotes), fall back to ast.literal_eval.
    """
    start = result_str.find('{')
    end = result_str.rfind('}') + 1
    if start == -1 or end == -1:
        raise ValueError("No JSON object found in the output.")
    json_str = result_str[start:end]
    try:
        return json.loads(json_str)
    except Exception:
        try:
            return ast.literal_eval(json_str)
        except Exception as e2:
            raise ValueError(f"Error parsing extraction result: {e2}")


In [63]:


# Example user input.
# user_input = (
#     "I deposited 5 million Tomans for 6 months. Also, I took a loan of 250000000 from Bank X. "
#     "My credit score is A. I have 12 installments "
# )


# user_input = (
#     "من میخوام یه وام ۳۵ میلیونی بگیرم. کلا هم میتونم توی اقساط ۸ ماهه پرداختش کنم"
# )

user_input = (
    "من ۲۰۰۰۰۰۰ پول توی ۴ ماه گذاشتم تو حسابم. الان میخوام ببینم چقدر وام ۴ درصد میتونم بگیرم. رتبه اعتباریم رو هم نمیدونم"
)

# user_input = (
#     "من میخوام یه وام ۳۵ میلیونی بگیرم. کلا هم میتونم تعداد ۸ قسط پرداختش کنم"
# )



# and the interest rate is 4.
# Run the extraction chain.
extraction_result = extraction_chain.run({"user_input": user_input})
print("Extraction result:")
print(extraction_result)

# Function to parse the LLM output into a Python dictionary.
def parse_output(result_str: str) -> dict:
    # Find the first '{' and the last '}' in the output.
    start = result_str.find('{')
    end = result_str.rfind('}') + 1
    if start == -1 or end == -1:
        raise ValueError("No JSON object found in the output.")
    json_str = result_str[start:end]
    try:
        return json.loads(json_str)
    except Exception:
        # Fallback to ast.literal_eval if JSON parsing fails.
        return ast.literal_eval(json_str)

# Parse the extracted output.
parsed_params = parse_output(extraction_result)
print("Parsed parameters:")
print(parsed_params)




> Entering new LLMChain chain...
Prompt after formatting:
Given the following user input:
"من ۲۰۰۰۰۰۰ پول توی ۴ ماه گذاشتم تو حسابم. الان میخوام ببینم چقدر وام ۴ درصد میتونم بگیرم. رتبه اعتباریم رو هم نمیدونم"

Extract the following parameters as a JSON object:
- deposit_amount: The amount of money deposited (if mentioned; otherwise, None).
- deposit_duration: The duration of the deposit in months (if mentioned; otherwise, None).
- loan_amount: The total loan amount (if mentioned; otherwise, None).
- credit_score (if mentioned; otherwise, None).
- number_of_installments (if mentioned; otherwise, None).
- interest_rate (if mentioned; otherwise, None).

If a parameter is not mentioned in the input, set its value to None.

Example output:
{
  "deposit_amount": 5000000,
  "deposit_duration": 6,
  "loan_amount": "None",
  "credit_score": "None",
  "number_of_installments": "None",
  "interest_rate": "None"
}

> Finished chain.
Extraction result:
To extract the parameters from the given us

## Check validation of parameters' value 

In [64]:
# def validate_parameters(extracted_params: dict) -> dict:
#     """
#     Validate each parameter value from extracted_params.
#     For each parameter:
#       - If a value is present but invalid, inform the user that the value is not valid.
#       - Prompt the user to enter a new value or press Enter to leave it as None.
#       - Update the dictionary with the valid (or None) value.
#     """
#     # Define valid criteria as lambda functions.
#     valid_criteria = {
#         "deposit_amount": lambda x: True,  # No limitations.
#         "deposit_duration": lambda x: 1 <= x <= 12,
#         "loan_amount": lambda x: x <= 300000000,
#         "credit_score": lambda x: x in ["A", "B", "C", "D", "E", "None"],
#         "number_of_installments": lambda x: x in [4, 6, 9, 12, 18, 24, 36, 48],
#         "interest_rate": lambda x: x in [0, 2, 4, 18, 23]
#     }
    
#     # Define user hints.
#     hints = {
#         "deposit_amount": "Enter any number (no limitations).",
#         "deposit_duration": "Enter an integer between 1 and 12 (months).",
#         "loan_amount": "Enter an integer no more than 300000000.",
#         "credit_score": "Enter one of: A, B, C, D, E, None.",
#         "number_of_installments": "Allowed values: 4, 6, 9, 12, 18, 24, 36, 48.",
#         "interest_rate": "Allowed values: 0, 2, 4, 18, 23."
#     }
    
#     for param, criteria in valid_criteria.items():
#         # Get the current value.
#         value = extracted_params.get(param)
        
#         # Convert the value to the appropriate type if not None.
#         if value is not None:
#             try:
#                 if param in ["deposit_duration", "number_of_installments", "interest_rate"]:
#                     current_val = int(value)
#                 elif  param in ["loan_amount", "deposit_amount"]:
#                     current_val = float(value)
#                 else:
#                     current_val = str(value).upper()
#             except Exception:
#                 current_val = None
#         else:
#             current_val = None
        
#         # If the value is valid, we keep it; otherwise, prompt the user.
#         if current_val is not None and criteria(current_val):
#             extracted_params[param] = current_val
#             new_exist_param = input(f"The extracted value for {param} is {extracted_params[param]}. Press Enter to keep it, or enter a new value: ")
#             if new_exist_param.strip():
#                 extracted_params[param] = float(new_exist_param)
#         else:
#             if current_val is not None:
#                 new_input = input(f"I got your value for '{param}' as {current_val}, which is not valid. {hints[param]}. Please enter a valid value for '{param}', or press Enter to set it as None: ").strip()
#                 print(f"I got your value for '{param}' as {current_val}, which is not valid. {hints[param]}")
#             else:
#                 print(f"No valid value was extracted for '{param}'. {hints[param]}")
#                 new_input = input(f"No valid value was extracted for '{param}'. {hints[param]}. Please enter a valid value for '{param}', or press Enter to set it as None: ").strip()
            
#             # Prompt the user.
            
#             if new_input == "":
#                 extracted_params[param] = None
#             else:
#                 try:
#                     if param in ["deposit_duration", "number_of_installments", "interest_rate"]:
#                         new_val = int(new_input)
#                     elif param in ["loan_amount", "deposit_amount"]:
#                         new_val = float(new_input)
#                     else:
#                         new_val = new_input.upper()
#                 except Exception:
#                     print(f"Could not convert your input for '{param}'. It will be set to None.")
#                     new_val = None
                
#                 # Check if the new value is valid.
#                 if new_val is not None and criteria(new_val):
#                     extracted_params[param] = new_val
#                 else:
#                     print(f"Your provided value for '{param}' is not valid. Setting it to None.")
#                     extracted_params[param] = None
#     return extracted_params

# # # Validate (and possibly override) the extracted parameters.
# # final_params = validate_parameters(extracted_params)
# # print("Final Validated Parameters:")
# # print(final_params)

In [65]:
def validate_parameters(extracted_params: dict) -> dict:
    """
    Validate each parameter value from extracted_params.
    For each parameter:
      - If a value is present and valid, prompt the user whether to keep or override it.
      - If the value is invalid or missing, repeatedly prompt the user until a valid value is provided or they press Enter (to set the value to None).
    """
    # Define valid criteria as lambda functions.
    valid_criteria = {
        "deposit_amount": lambda x: True,  # No limitations.
        "deposit_duration": lambda x: 1 <= x <= 12,
        "loan_amount": lambda x: x <= 300000000,
        "credit_score": lambda x: x in ["A", "B", "C", "D", "E", "None"],
        "number_of_installments": lambda x: x in [4, 6, 9, 12, 18, 24, 36, 48],
        "interest_rate": lambda x: x in [0, 2, 4, 18, 23]
    }
    
    # Define user hints.
    hints = {
        "deposit_amount": "Enter any number (no limitations).",
        "deposit_duration": "Enter an integer between 1 and 12 (months).",
        "loan_amount": "Enter an integer no more than 300000000.",
        "credit_score": "Enter one of: A, B, C, D, E, None.",
        "number_of_installments": "Allowed values: 4, 6, 9, 12, 18, 24, 36, 48.",
        "interest_rate": "Allowed values: 0, 2, 4, 18, 23."
    }
    
    for param, criteria in valid_criteria.items():
        # Get current extracted value.
        value = extracted_params.get(param)
        if value is not None:
            try:
                if param in ["deposit_duration", "number_of_installments", "interest_rate"]:
                    current_val = int(value)
                elif param in ["loan_amount", "deposit_amount"]:
                    current_val = float(value)
                else:
                    current_val = str(value).upper()
            except Exception:
                current_val = None
        else:
            current_val = None

        # If a valid value is present, ask if the user wants to keep or override it.
        if current_val is not None and criteria(current_val):
            user_input = input(
                    f"The extracted value for '{param}' is {current_val}. Press Enter to keep it, "
                    f"or enter a new value. (Hint: {hints[param]}). "
                    f"Press Enter without input to keep the value, or type a new value: "
                ).strip()
            while True:
                # If user presses Enter, keep current value.
                if user_input == "":
                    extracted_params[param] = current_val
                    break
                else:
                    try:
                        if param in ["deposit_duration", "number_of_installments", "interest_rate"]:
                            candidate_val = int(user_input)
                        elif param in ["loan_amount", "deposit_amount"]:
                            candidate_val = float(user_input)
                        else:
                            candidate_val = user_input.upper()
                    except Exception:
                        print(f"Could not convert your input for '{param}'. Please try again.")
                        continue
                    if criteria(candidate_val):
                        extracted_params[param] = candidate_val
                        break
                    else:
                        user_input = input(f"I got your value for '{param}' as {candidate_val}, which is not valid. {hints[param]}. Please enter a valid value for '{param}', or press Enter to set it as None: ").strip()
                        # print(f"I got your value for '{param}' as {current_val}, which is not valid. {hints[param]}")
                        
                        print(f"Your provided value '{candidate_val}' for '{param}' is not valid. {hints[param]}.")
                        # Continue prompting until valid or blank.
        else:
            if current_val is not None:
        
                user_input = input(
                        f"I got your value for '{param}' as {current_val}, which is not valid. {hints[param]}." 
                        f"Please enter a valid value for '{param}', or press Enter to set it as None: "
                        ).strip()
                while True:
                    if user_input == "":
                        extracted_params[param] = None
                        break
                    else:
                        try:
                            if param in ["deposit_duration", "number_of_installments", "interest_rate"]:
                                candidate_val = int(user_input)
                            elif param in ["loan_amount", "deposit_amount"]:
                                candidate_val = float(user_input)
                            else:
                                candidate_val = user_input.upper()
                        except Exception:
                            print(f"Could not convert your input for '{param}'. Please try again.")
                            continue
                        if criteria(candidate_val):
                            extracted_params[param] = candidate_val
                            break
                        else:
                            user_input = input(f"I got your value for '{param}' as {candidate_val}, which is not valid. {hints[param]}. Please enter a valid value for '{param}', or press Enter to set it as None: ").strip()
                            print(f"Your provided value '{candidate_val}' for '{param}' is not valid. {hints[param]}.")
        
    
            else:
                # No valid value is present.
                user_input = input(
                        f"No valid value was extracted for '{param}'. {hints[param]}. "
                        f"Please enter a valid value for '{param}', or press Enter to set it to None: "
                    ).strip()
                while True:
                    if user_input == "":
                        extracted_params[param] = None
                        break
                    else:
                        try:
                            if param in ["deposit_duration", "number_of_installments", "interest_rate"]:
                                candidate_val = int(user_input)
                            elif param in ["loan_amount", "deposit_amount"]:
                                candidate_val = float(user_input)
                            else:
                                candidate_val = user_input.upper()
                        except Exception:
                            print(f"Could not convert your input for '{param}'. Please try again.")
                            continue
                        if criteria(candidate_val):
                            extracted_params[param] = candidate_val
                            break
                        else:
                            user_input = input(f"I got your value for '{param}' as {candidate_val}, which is not valid. {hints[param]}. Please enter a valid value for '{param}', or press Enter to set it as None: ").strip()
                            print(f"Your provided value '{candidate_val}' for '{param}' is not valid. {hints[param]}.")
                            # Loop until valid input is provided or user presses Enter.
    return extracted_params


In [66]:
# Example user input (this could be from a chatbot, web form, etc.).
# user_input = (
#     "I deposited 5 million Tomans for 6 months. I took a loan of 250000000 from Bank X. "
#     "My credit score is A. I have 12 installments and the interest rate is 4."
# )

# user_input = (
#     "من میخوام یه وام ۳۵ میلیونی بگیرم. کلا هم میتونم توی اقساط ۸ ماهه پرداختش کنم"
# )

user_input = (
    "من ۲۰۰۰۰۰۰ پول توی ۴ ماه گذاشتم تو حسابم. الان میخوام ببینم چقدر وام ۳ درصد میتونم بگیرم. رتبه اعتباریم رو هم نمیدونم"
)

# Use the LLM extraction chain to get parameters from the input.
extraction_result = extraction_chain.run({"user_input": user_input})
print("LLM Extraction Result:")
print(extraction_result)

# Parse the extraction result to a Python dictionary.
extracted_params = parse_extraction_result(extraction_result)
print("Extracted Parameters (from LLM):")
print(extracted_params)

# Validate or prompt the user for any missing/incorrect parameters.
final_params = validate_parameters(extracted_params)
# final_params = double_check_parameters(extracted_params)
print("Final Validated Parameters:")
print(final_params)



> Entering new LLMChain chain...
Prompt after formatting:
Given the following user input:
"من ۲۰۰۰۰۰۰ پول توی ۴ ماه گذاشتم تو حسابم. الان میخوام ببینم چقدر وام ۳ درصد میتونم بگیرم. رتبه اعتباریم رو هم نمیدونم"

Extract the following parameters as a JSON object:
- deposit_amount: The amount of money deposited (if mentioned; otherwise, None).
- deposit_duration: The duration of the deposit in months (if mentioned; otherwise, None).
- loan_amount: The total loan amount (if mentioned; otherwise, None).
- credit_score (if mentioned; otherwise, None).
- number_of_installments (if mentioned; otherwise, None).
- interest_rate (if mentioned; otherwise, None).

If a parameter is not mentioned in the input, set its value to None.

Example output:
{
  "deposit_amount": 5000000,
  "deposit_duration": 6,
  "loan_amount": "None",
  "credit_score": "None",
  "number_of_installments": "None",
  "interest_rate": "None"
}

> Finished chain.
LLM Extraction Result:
To extract the required parameters from